This code crawls aroung linkedin according to the list of your contacts supplied. It stores user information: name, company name, title, employement dates, school attended, field of study, study dates. This information gathered for each user crawled and stored into three separated relational tables: name, education, company.

In [ ]:
# Dependencies
import mysql.connector
from mysql.connector import errorcode
from datetime import date, datetime, timedelta
from scrape_linkedin import ProfileScraper
from selenium import webdriver
import time
import re
from transliterate import translit
from unidecode import unidecode
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import string
import re        

In [ ]:
#functions in use

# Adds user names to the corresponding table
def user_adder(user_id, personal_info):
    
    add_user = ("INSERT INTO name "
               "(id_name, name) "
               "VALUES (%s, %s)")
    
    RE = re.compile(u'[⺀-⺙⺛-⻳⼀-⿕々〇〡-〩〸-〺〻㐀-䶵一-鿃豈-鶴侮-頻並-龎]', re.UNICODE)
    
    if personal_info['name'] is not None:
        personal_info['name'] = RE.sub('', personal_info['name'])
        if len(personal_info['name']) == 0:
            personal_info['name'] = "Chinese Name"
        if has_cyrillic(personal_info['name']):
            personal_info['name'] = translit(personal_info['name'], "ru", reversed=True)
    
        personal_info['name'] = unidecode(personal_info['name'])
    
    personal_info['name'] = ['None' if v is None else v for v in [personal_info['name']]]
    personal_info['name'] = ''.join(personal_info['name'])   
    print(personal_info['name'])
    data_user = (user_id, personal_info['name'])
    
    cursor.execute(add_user, data_user)
    
    cnx.commit()

# Adds a company name, title, dates of employement to the corresponding table
def companies_adder(user_id, list_of_jobs):
    
    for info_block in list_of_jobs:

        add_company = ("INSERT INTO company "
                   "(id_name, company, title, date_from, date_to) "
                   "VALUES (%s, %s, %s, %s, %s)")
        
        if info_block['date_range'] is not None:
            
            if ('–' in info_block['date_range'] and 'Present' not in info_block['date_range']):
                
                date_time_str_from = info_block['date_range'].split('–')[0].rstrip()
                if len(date_time_str_from) == 4:
                    date_time_obj_from = datetime.strptime(date_time_str_from, '%Y').date()
                else:
                    date_time_obj_from = datetime.strptime(date_time_str_from, '%b %Y').date()

                date_time_str_to = info_block['date_range'].split('–')[1].lstrip()
                if len(date_time_str_to) == 4:
                    date_time_obj_to = datetime.strptime(date_time_str_to, '%Y').date()
                else:
                    date_time_obj_to = datetime.strptime(date_time_str_to, '%b %Y').date()
                

            if ('–' in info_block['date_range'] and 'Present' in info_block['date_range']):

                date_time_str_from = info_block['date_range'].split('–')[0].rstrip()
                if len(date_time_str_from) == 4:
                    date_time_obj_from = datetime.strptime(date_time_str_from, '%Y').date()
                else:
                    date_time_obj_from = datetime.strptime(date_time_str_from, '%b %Y').date()

                date_time_obj_to = str(date(2115, 1, 1))

            if '–' not in info_block['date_range']:
                
                date_time_str_from = info_block['date_range']
                if len(date_time_str_from) == 4:
                    date_time_obj_from = datetime.strptime(date_time_str_from, '%Y').date()
                else:
                    date_time_obj_from = datetime.strptime(date_time_str_from, '%b %Y').date()
                

                date_time_obj_to = date_time_obj_from
        
        else: 
            
            date_time_obj_from = date.min
            date_time_obj_to = date.min
        
        RE = re.compile(u'[⺀-⺙⺛-⻳⼀-⿕々〇〡-〩〸-〺〻㐀-䶵一-鿃豈-鶴侮-頻並-龎]', re.UNICODE)
        
        if info_block['company'] is not None:
            info_block['company']= RE.sub('', info_block['company'])
            if has_cyrillic(info_block['company']):
                info_block['company'] = translit(info_block['company'], "ru", reversed=True)
            info_block['company'] = unidecode(info_block['company'])
        
        if info_block['title'] is not None:
            info_block['title'] = RE.sub('', info_block['title'])
            if has_cyrillic(info_block['title']):
                info_block['title'] = translit(info_block['title'], "ru", reversed=True)
            info_block['title'] = unidecode(info_block['title'])
        
        info_block['company'], info_block['title'], date_time_obj_from, date_time_obj_to = ['None' if v is None else v for v in [info_block['company'], 
                                                                                         info_block['title'], 
                                                                                         date_time_obj_from, 
                                                                                         date_time_obj_to]] 
        data_company = (user_id, info_block['company'], info_block['title'], date_time_obj_from, date_time_obj_to)

        cursor.execute(add_company, data_company)

    cnx.commit()

# Adds education: school, field of study, dates attended 
def education_adder(user_id, list_of_edu):

    for info_block in list_of_edu:

        add_edu = ("INSERT INTO education "
                   "(id_name, school, field_of_study, date_from, date_to) "
                   "VALUES (%s, %s, %s, %s, %s)")
        
        if info_block['date_range'] is not None:
        
            if ('–' in info_block['date_range'] and 'Present' not in info_block['date_range']):

                date_time_str_from = info_block['date_range'].split('–')[0].rstrip()
                if len(date_time_str_from) == 4:
                    date_time_obj_from = datetime.strptime(date_time_str_from, '%Y').date()
                else:
                    date_time_obj_from = datetime.strptime(date_time_str_from, '%b %Y').date()
                

                date_time_str_to = info_block['date_range'].split('–')[1].lstrip()
                if len(date_time_str_to) == 4:
                    date_time_obj_to = datetime.strptime(date_time_str_to, '%Y').date()
                else:
                    date_time_obj_to = datetime.strptime(date_time_str_to, '%b %Y').date()
                

            if ('–' in info_block['date_range'] and 'Present' in info_block['date_range']):

                date_time_str_from = info_block['date_range'].split('–')[0].rstrip()
                if len(date_time_str_from) == 4:
                    date_time_obj_from = datetime.strptime(date_time_str_from, '%Y').date()
                else:
                    date_time_obj_from = datetime.strptime(date_time_str_from, '%b %Y').date()

                date_time_obj_to = str(date(2115, 1, 1))

            if '–' not in info_block['date_range']:

                date_time_str_from = info_block['date_range']
                if len(date_time_str_from) == 4:
                    date_time_obj_from = datetime.strptime(date_time_str_from, '%Y').date()
                else:
                    date_time_obj_from = datetime.strptime(date_time_str_from, '%b %Y').date()

                date_time_obj_to = date_time_obj_from
                
        else: 
            date_time_obj_from = date.min
            date_time_obj_to = date.min
        
        RE = re.compile(u'[⺀-⺙⺛-⻳⼀-⿕々〇〡-〩〸-〺〻㐀-䶵一-鿃豈-鶴侮-頻並-龎]', re.UNICODE)
        
        if info_block['name'] is not None:
            info_block['name']= RE.sub('', info_block['name'])
            if has_cyrillic(info_block['name']):
                info_block['name'] = translit(info_block['name'], "ru", reversed=True)
            info_block['name'] = unidecode(info_block['name'])
        
        if info_block['field_of_study'] is not None:
            info_block['field_of_study'] = RE.sub('', info_block['field_of_study'])
            if has_cyrillic(info_block['field_of_study']):
                info_block['field_of_study'] = translit(info_block['field_of_study'], "ru", reversed=True)
            info_block['field_of_study'] = unidecode(info_block['field_of_study'])
        
        info_block['name'], info_block['field_of_study'], date_time_obj_from, date_time_obj_to = ['None' if v is None else v for v in [info_block['name'], 
                                                                                                 info_block['field_of_study'], 
                                                                                                 date_time_obj_from, 
                                                                                                 date_time_obj_to]] 
        data_edu = (user_id, info_block['name'], info_block['field_of_study'], date_time_obj_from, date_time_obj_to)

        cursor.execute(add_edu, data_edu)

    cnx.commit()
    
def has_cyrillic(text):
    return bool(re.search('[\u0400-\u04FF]', text))


def cleaner(raw_list):
    clean_names = []

    emoji_pattern = re.compile(u"[^\U00000000-\U0000d7ff\U0000e000-\U0000ffff]", flags=re.UNICODE)

    for name in raw_list:
        

        item = re.sub(r" ?\([^)]+\)", "", name)
        item = emoji_pattern.sub(r'', item)
        clean_names.append(item)
    return clean_names

def rawList(path):
    
    data = pd.read_csv(path)

    name_list_raw = []
    for i, j in data.iterrows(): 

        if type(j['First Name']) is str and type(j['Last Name']) is str:
            name_list_raw.append(' '.join([j['First Name'],j['Last Name']]))
    print("raw_list is formed")
    
    return name_list_raw

def extractNames(login, password, raw_list, PATH_TO_YOUR_LINKEDIN_PAGE):

    driver = webdriver.Chrome()
    driver.get(PATH_TO_YOUR_LINKEDIN_PAGE)

    time.sleep(0.5)

    driver.find_element_by_xpath('//*[@id="join-form"]/p[3]/a').click()

    time.sleep(0.5)

    username = driver.find_element_by_id("login-email")

    username.send_keys(login)

    time.sleep(0.5)

    password_web = driver.find_element_by_id("login-password")
  
    password_web.send_keys(password)

    time.sleep(0.5)


    driver.find_element_by_id("login-submit").click()

    time.sleep(5)
    

    c = 0
    for raw_name in raw_list:

        print(c, raw_name, "is extracting")
        search = driver.find_element_by_xpath("//input[contains(@placeholder, 'Search')]")

        search.clear()
        search.send_keys(raw_name)

        time.sleep(0.5)

        search.send_keys(Keys.ENTER)

        time.sleep(2)
       
        name = driver.find_elements_by_xpath("//a[contains(@data-control-name, 'search_srp_result')]")[0].get_attribute("href").split("/")[-2]
        time.sleep(0.5)
        name_list_linkedin.append(name)
        c += 1
        
    print("extraction is done")
    
    time.sleep(0.5)
    
    driver.close()

In [ ]:
# connect to db
cnx = mysql.connector.connect(user='YOUR USER_NAME', password='YOUR_PASSWORD',
                              host='YOUR HOST_IP',
                              database='DB_NAME')
cursor = cnx.cursor()
print('Connection', cnx.is_connected())

In [ ]:
#Table initialization
# Creates tables in yr database

TABLES = {}
TABLES['name'] = (
    "CREATE TABLE `name` ("
    "  `id_name` int(11) NOT NULL,"
    "  `name` varchar(100) NOT NULL,"
    "  PRIMARY KEY (`id_name`)"
    ")")

TABLES['education'] = (
    "CREATE TABLE `education` ("
    "  `id_edu` INT NOT NULL AUTO_INCREMENT,"
    "  `id_name` int(11) NOT NULL,"
    "  `school` varchar(100),"
    "  `field_of_study` varchar(100),"
    "  `date_from` date,"
    "  `date_to` date,"
    "  PRIMARY KEY (`id_edu`),"
    "  CONSTRAINT `fk_edu_to_names` FOREIGN KEY (`id_name`) "
    "     REFERENCES `name` (`id_name`) ON DELETE CASCADE ON UPDATE CASCADE)"
    )

TABLES['company'] = (
    "CREATE TABLE `company` ("
    "  `id_comp` INT NOT NULL AUTO_INCREMENT,"
    "  `id_name` int(11) NOT NULL,"  
    "  `company` varchar(100),"
    "  `title` varchar(100),"
    "  `date_from` date ,"
    "  `date_to` date ,"
    "  PRIMARY KEY (`id_comp`),"
    "  CONSTRAINT `fk_comp_to_names` FOREIGN KEY (`id_name`) "
    "     REFERENCES `name` (`id_name`) ON DELETE CASCADE ON UPDATE CASCADE)"
    )

for table_name in TABLES:
    desc = TABLES[table_name]
    try:
        print("Creating table {}: ".format(table_name), end='')
        cursor.execute(desc)
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
            print("already exists.")
        else:
            print(desc)
            print(err.msg)
    else:
        print("OK")
cnx.commit()

In [ ]:
# 1. download list of connections from linkedin
# 2. set a path to a csv file with connections
# 3. set your login and password

path = "YOUR PATH TO CSV"
login = 'your_login'
password = "your_password"

name_list_linkedin = []
name_list_raw = rawList(path)
name_list_raw_clean = cleaner(name_list_raw)

PATH_TO_YOUR_LINKEDIN_PAGE = "https://www.linkedin.com/in/YOUR_USER_NAME/"

extractNames(login, password, name_list_raw_clean, PATH_TO_YOUR_LINKEDIN_PAGE)

In [ ]:
#write out prepared names to txt for further use
PATH_TO_STORED_NAMELIST = "YOUR_PATH"
with open(PATH_TO_STORED_NAMELIST, "w") as MyFile: # Use file to refer to the file object
    for element in name_list_linkedin:
        MyFile.write(element)
        MyFile.write('\n')

In [ ]:
#read prepared names to python list for tranfer to main function of extraction from linkedin to DB
name_list_linkedin = []

PATH_TO_STORED_NAMELIST = "YOUR_PATH"

with open(PATH_TO_STORED_NAMELIST, "r") as MyFile: # Use file to refer to the file object
    for name in MyFile:
        name_list_linkedin.append(name.strip())

In [ ]:
# 0.
# Install from source
# git clone https://github.com/austinoboyle/scrape-linkedin-selenium.git
# Run python setup.py install

# 1.
# Getting LI_AT
# 1.Navigate to www.linkedin.com and log in
# 2.Open browser developer tools (Ctrl-Shift-I or right click -> inspect element)
# 3.Select the appropriate tab for your browser (Application on Chrome, Storage on Firefox)
# 4.Click the Cookies dropdown on the left-hand menu, and select the www.linkedin.com option
# 5.Find and copy the li_at value

# 2.
# download chromedriver and set the path as below
# driver = webdriver.Chrome(YOUR_PATH)


user_list = name_list_linkedin

#calculate current rows in the table name
cursor.execute("select count(*) from name")
n = cursor.fetchone()
n = n[0]

#main process of uploading to DB
with ProfileScraper(cookie='YOUR_LIAT') as scraper:
    print('uploaded before', n)
    
    for user_id in range(len(user_list)):
#         try:
        print('this number is', user_id+n+1, user_list[user_id])
        print('retrieve from linkedin')
        profile = scraper.scrape(user=user_list[user_id])

        user_adder(user_id+n+1, profile.personal_info)
        print('upload user name to DB')
        companies_adder(user_id+n+1, profile.experiences['jobs'])
        print('upload jobs name to DB')
        education_adder(user_id+n+1, profile.experiences['education'])
        print('upload education name to DB')
#         except:
#             continue
            
        time.sleep(5)
        
print('done')
# print (profile.experiences.keys())

In [ ]:
#close connections
cursor.close()
cnx.close()